<a href="https://colab.research.google.com/github/StephMcCallum/CHEM561/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 - Coupled Oscillators

**Overview**

This notebook guides you through the simulation of multiple harmonic systems coupled together. The notebook will analyze a molecule composed by three atoms linked by two bonds. As the mass of the central atom is varied, the two bonds will become more or less coupled, giving rise to a motion that seems to flow from one bond to the other. However, when analyzed using linear algebra (eingenvalues and eigenvectors), the motion reverts to the dynamics of two independent harmonic oscillators (normal modes).

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
import math
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

**Problem**

Let's assume that we want to model the motion of small rigid (and highly symmetric) molecules, such as carbon dioxide (CO2), triiodide (I3-), cianide (HCN). At low temperature the energy associated with the bonds in these molecules has very deep minima and it is reasonable to assume the motion of each bond is going to be harmonic. However, when we consider both bonds at the same time, the motions of the individual atoms may become less easy to interpret. We want to understand how the relative masses of the atoms involved in the bonds affect the localized or delocalized nature of the molecular vibrations.

**Model**

We are going to assume that the atoms only move in the axial direction (1D problem) and we are going to model each bond using a classical spring. We will integrate the equations of motion using Velocity Verlet with a small timestep. By looking at systems with different masses, symmetric and asymmetric, we aim to understand how well the motions of the individual springs are preserved when they are combined together.

> Is the coupling between springs always going to be important? Can we still treat systems of coupled harmonic oscillators as a collection of independent vibrations?

To answer some of these questions, we will consider an ever simpler model, in which the central atom of the molecule is kept fixed by assigning it an infinite mass.

**Questions**

Before you run any simulation, answer the following question(s):

1. Considering you have a system of 3 atoms moving in 1 dimension, how many vibrations do you expect to see?
2. If the central atom is fixed, do you expect that the vibrations of the two bonds will affect each other? Considering a simplified problem where all the relevant quantities have unit values, what would be the frequencies of vibration of the two bonds?
3. In a problem where two atoms have masses significantly larger than the third (such as in HCN), do you think that the motion of the two bonds will be more or less affected by the fact they are connected through the central atom?   

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

4. Start with a system where the central atom is fixed. Consider both symmetric and asymmetric molecules (i.e. masses of the side atoms). Are the motions of the atoms coupled? Is there any potential energy moving from one side of the molecule to the other?
5. Now consider a symmetric system, but progressively reduce the mass of the central atom. When the three masses are identical you have a good model for I3-. How are the vibrations of the molecule, in particular their frequencies, changing? Would you say that reducing the mass makes the system more or less coupled?
6. Can you find any relation between the diagonal part of the dynamic matrix and the final frequencies of the vibrational modes?
7. Now consider an asymmetric system, where one of the atoms has a significantly smaller mass than the other two. Do you think this system is more or less 'coupled' than the symmetric system (i.e. the final trajectories and frequencies look more or less similar to the ones of two indipendend harmonic oscillators)?

In [ ]:
# @title Utility Functions { display-mode: "form" }
# --- functions to compute forces and energies ---

def compute_kinetic(velocities, masses):
    """
    velocities: (N,2) array of particle velocities
    masses:     (N,) array of particle masses
    """
    # Multiply each particle's speed squared by its mass
    v2 = np.sum(velocities**2, axis=1)  # sum over x and y
    KE = 0.5 * np.sum(masses * v2)
    return KE

def compute_potential(positions, springs):
    """
    positions: (N,2) array of particle positions
    springs:   list of (i,j,props) with props={"k":..., "L0":...}
    """
    U = 0.0
    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]
        # vector displacement
        rij = positions[i] - positions[j]
        dist = np.linalg.norm(rij)   # Euclidean distance in 2D
        # potential energy of spring
        U += 0.5 * k * (dist - L0)**2
    return U

def compute_forces(nparticles, positions, springs):
    """
    Compute forces on each particle due to springs in 2D.

    nparticles : int
        Number of particles
    positions : (N,2) array
        Particle positions
    springs : list of (i,j,props)
        Each spring connects particles i and j with props={"k":..., "L0":...}
    """
    forces = np.zeros((nparticles, 2))  # 2D forces

    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]

        # vector displacement
        rij = positions[i] - positions[j]
        dist = np.linalg.norm(rij)
        if dist == 0:
            continue  # avoid division by zero

        # Hooke's law in 2D
        F = -k * (dist - L0) * (rij / dist)

        # Apply equal and opposite forces
        forces[i] += F
        forces[j] -= F

    return forces

def compute_hessian(nparticles, springs):
    """
    Build a 2D Hessian matrix for nparticles with springs,
    without needing positions (static graph Laplacian per DOF).

    nparticles : int
        Number of particles
    springs : list of (i, j, props)
        Each spring is (i, j, {"k": spring constant, "L0": rest length})
    """
    ndof = 2 * nparticles
    H = np.zeros((ndof, ndof))

    for (i, j, props) in springs:
        k = props["k"]

        # for both x and y DOF
        for a in range(2):  # 0=x, 1=y
            ii, jj = 2*i+a, 2*j+a
            H[ii, ii] += k
            H[jj, jj] += k
            H[ii, jj] -= k
            H[jj, ii] -= k

    return H


def compute_dynamic_matrix(nparticles, masses, springs):
    """
    Build 2D dynamic matrix (mass-weighted Hessian).
    No positions required.

    nparticles : int
        Number of particles
    masses : list/array of length N
    springs : list of (i, j, props)
    """
    ndof = 2 * nparticles
    H = compute_hessian(nparticles, springs)
    D = np.zeros((ndof, ndof))

    # mass vector (repeat for x and y DOF)
    M = np.zeros(ndof)
    for i in range(nparticles):
        M[2*i]   = masses[i]
        M[2*i+1] = masses[i]

    for i in range(ndof):
        for j in range(ndof):
            D[i, j] = H[i, j] / np.sqrt(M[i] * M[j])

    return D

In [ ]:
# @title System Setup  { display-mode: "form" }
AB_eq_bond_length = 1.0
AC_eq_bond_length = 1.0
AB_bond_strenght = 50 # @param {type:"number"}
AC_bond_strenght = 50 # @param {type:"number"}
ABC_init_angle = 1.82
ABC_bond_strenght = 100 # @param {type:"number"}
ABC_eq_bond_length = 2*math.sin(ABC_init_angle/2)
A_mass = "very large" # @param ["unit", "double", "large", "very large", "infinite"]
B_mass = 1  # @param {type:"number"}
C_mass = 1  # @param {type:"number"}

In [ ]:
# @title Static Analysis of the System  { display-mode: "form" }

mass_map = {
    "unit": 1.0,
    "double": 2.0,
    "large": 10.0,
    "very large": 20.0,
    "infinite": 1e10   # effectively pinned
}

# Initial conditions
A_pos = np.zeros(2)
B_pos = A_pos + np.array([AB_eq_bond_length*math.sin(ABC_init_angle/2),AB_eq_bond_length*math.cos(ABC_init_angle/2)])
C_pos = A_pos - np.array([AC_eq_bond_length*math.sin(ABC_init_angle/2),-1*AC_eq_bond_length*math.cos(ABC_init_angle/2)])

plt.figure(figsize=(5,5))
for (x, y, label) in [(A_pos[0], A_pos[1], 'A'),
                      (B_pos[0], B_pos[1], 'B'),
                      (C_pos[0], C_pos[1], 'C')]:
    plt.scatter(x, y, s=100)
    plt.text(x+0.05, y+0.05, label, fontsize=12)

# Draw bonds
plt.plot([A_pos[0], B_pos[0]], [A_pos[1], B_pos[1]], 'k-')
plt.plot([A_pos[0], C_pos[0]], [A_pos[1], C_pos[1]], 'k-')

plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("x")
plt.ylabel("y")
plt.title("2D 3-Atom Geometry (A-B-C)")
plt.grid(True)
plt.show()

# -- Define the system --
nparticles = 3
positions = np.array([B_pos, A_pos, C_pos])
masses = np.array([B_mass, mass_map[A_mass], C_mass])
print(positions, masses)
nspring = 3
springs = [
    (0, 1, {"k": AB_bond_strenght, "L0": AB_eq_bond_length}),
    (0, 2, {"k": ABC_bond_strenght, "L0": ABC_eq_bond_length}),
    (1, 2, {"k": AC_bond_strenght, "L0": AC_eq_bond_length})
]
# -- Compute the Dynamic Matrix of the system --
D = compute_dynamic_matrix(nparticles, masses, springs)
print(D)

# -- Diagonalize the Dynamic Matrix --
eigenvalues, eigenvectors = np.linalg.eigh(D)

# Diagonal matrix from eigenvalues
Lambda = np.diag(eigenvalues)

# Example equilibrium positions (triangle)
positions = np.array([[0,0], [1,0], [0.5,0.866]])  # shape (3,2)
nparticles = positions.shape[0]

# Frequencies
frequencies = np.sqrt(np.abs(eigenvalues))

# --- Figure with a 2x2 grid ---
fig = plt.figure(figsize=(12, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 2, width_ratios=[1, 1.5])  # left: matrices, right: modes

# --- Top-left: Dynamic matrix D ---
ax1 = fig.add_subplot(gs[0, 0])
norm = TwoSlopeNorm(vmin=min(D.min(), Lambda.min()) - 1e-1,
                    vcenter=0,
                    vmax=max(D.max(), Lambda.max()) + 1e-1)
cax1 = ax1.matshow(D, cmap="seismic", norm=norm)
ax1.set_title("Dynamic Matrix D")
for (i, j), val in np.ndenumerate(D):
    ax1.text(j, i, f"{val:.2f}", va="center", ha="center", color="white", fontsize=8)

# --- Bottom-left: Diagonal matrix Λ ---
ax2 = fig.add_subplot(gs[1, 0])
cax2 = ax2.matshow(Lambda, cmap="seismic", norm=norm)
ax2.set_title("Diagonal Matrix (Eigenvalues)")
for (i, j), val in np.ndenumerate(Lambda):
    ax2.text(j, i, f"{val:.2f}", va="center", ha="center", color="white", fontsize=8)

# Shared colorbar
fig.colorbar(cax1, ax=[ax1, ax2], shrink=0.8)

# --- Right column: plot first 3 non-trivial modes ---
nmodes_to_show = 3
gs_right = gs[:, 1].subgridspec(nmodes_to_show, 1)

for mode in range(nmodes_to_show):
    ax = fig.add_subplot(gs_right[mode])

    vec = eigenvectors[:, mode]  # length = 6
    vec2d = vec.reshape((nparticles, 2))  # reshape into (3,2)

    # normalize displacement size
    scale = 0.3 * np.max(np.ptp(positions, axis=0))
    displacements = scale * vec2d / np.max(np.abs(vec2d))

    # plot equilibrium positions
    ax.scatter(positions[:,0], positions[:,1], s=200, c="black", zorder=2)

    # plot arrows for displacements
    for i in range(nparticles):
        ax.arrow(positions[i,0], positions[i,1],
                 displacements[i,0], displacements[i,1],
                 head_width=0.05, head_length=0.05,
                 fc="red", ec="red", length_includes_head=True, zorder=3)

    ax.set_title(f"Mode {mode+1}\nλ={eigenvalues[mode]:.3f}, ω={frequencies[mode]:.3f}")
    ax.set_aspect("equal")
    ax.grid(True)

plt.suptitle("Normal Mode Analysis (2D, 3 Particles)", fontsize=16)
plt.show()


In [ ]:
# @title Classical Dynamics Parameters  { display-mode: "form" }
Temperature = 0.1 # @param {type:"number"}
dt = 0.01  # @param {type:"number"}
nsteps = 10000  # @param {type:"integer"}
frame_stride = 500 # @param {type:"integer"}
total_time = nsteps * dt
random_start = True # @param {type:"boolean"}
remove_com = True # @param {type:"boolean"}

In [ ]:
# @title Run the Simulation  { display-mode: "form" }
# Initial conditions
positions = np.array([B_pos, A_pos, C_pos]) #add y dimension here
# Initialize normal mode velocities from equipartition
kB = 1.0  # use 1 in reduced units

# Initialize random or uniform modal velocities
if random_start:
    # 2 DOF per particle → shape (2N,)
    qdot = np.random.normal(0, np.sqrt(kB * Temperature), size=2*nparticles)
else:
    qdot = np.ones(2*nparticles) * np.sqrt(kB * Temperature)

# Kill zero-frequency modes (translations/rotations)
if remove_com:
    tol = 1e-5
    zero_modes = np.where(np.abs(eigenvalues) < tol)[0]
    qdot[zero_modes] = 0.0  # zero out these modal velocities

# Back-transform to particle velocities in DOF space
M = np.repeat(masses, 2)  # repeat for x and y
velocities_dof = (eigenvectors @ qdot) / np.sqrt(M)  # shape (2N,)

# Optional: reshape to (N,2) for easier handling
velocities = velocities_dof.reshape((nparticles, 2))

# -- Setup ---
kin = compute_kinetic(velocities, masses)
pot = compute_potential(positions, springs)
time = np.arange(0, total_time, dt)
trajectory = []
energy = []
# -- Loop over time ---
for t in time:
    trajectory.append(positions.copy())
    energy.append((kin, pot, kin + pot))

    forces = compute_forces(nparticles, positions, springs)
    # Ensure masses broadcast correctly for (N,2)
    masses_col = masses[:, np.newaxis]   # shape (N,1)

    positions += velocities * dt + 0.5 * (forces / masses_col) * dt**2
    new_forces = compute_forces(nparticles, positions, springs)
    velocities += 0.5 * ((forces + new_forces) / masses_col) * dt

    # Remove center of mass motion
    if remove_com:
        vcom = np.sum(velocities * masses_col) / np.sum(masses)
        velocities -= vcom

    kin = compute_kinetic(velocities, masses)
    pot = compute_potential(positions, springs)

# -- end of time loop --
trajectory = np.array(trajectory)  # shape (nt, nparticles)

# --- Mass-weighted projection into eigenmodes ---
nt, N, dim = trajectory.shape  # (nt, N, 2)
ndof = N * dim

# Flatten trajectory and equilibrium positions: (nt, 2N)
traj_flat = trajectory.reshape(nt, ndof)
eq_pos = trajectory[0]
eq_flat   = eq_pos.reshape(ndof)

# Mass vector repeated for x and y DOFs
M_sqrt = np.repeat(np.sqrt(masses), dim)  # shape (2N,)

# Apply mass-weighting
trajectory_massweighted = (traj_flat - eq_flat) * M_sqrt[np.newaxis, :]  # (nt, 2N)

# Project onto eigenmodes
q_traj = trajectory_massweighted @ eigenvectors   # (nt, nmodes)

# --- Identify and drop the translation modes (2 in 2D) ---
mode_order = np.argsort(eigenvalues)   # ascending order
nontrivial_modes = mode_order[dim:]    # drop first `dim` modes (translations)

eigenvalues_nt = eigenvalues[nontrivial_modes]
frequencies_nt = np.sqrt(np.abs(eigenvalues_nt))
q_traj_nt = q_traj[:, nontrivial_modes]
nmodes = len(nontrivial_modes)


In [ ]:
# @title Visualize the Simulation (2D with Bonds) { display-mode: "form" }
colors = ["red", "blue", "green"]   # particle colors
bond_color = "gray"
cb_color   = "orange"  # special color for C–B bond

# --- Define bond list (must match your springs list) ---
# Example: A=0, B=1, C=2
bonds = [(0, 1), (0, 2), (1, 2)]   # AB, AC, CB

# --- Reconstruct particle trajectories from individual modes ---
M_isqrt = 1.0 / np.sqrt(np.repeat(masses, 2))   # shape (2N,)

def reconstruct_from_mode(q_mode, mode_vector):
    nt = q_mode.shape[0]
    displacements = q_mode[:, None] * (mode_vector[None, :] * M_isqrt[None, :])  # (nt, 2N)
    return eq_pos.reshape(-1) + displacements   # (nt, 2N)

x_from_mode1 = reconstruct_from_mode(q_traj_nt[:, 0], eigenvectors[:, nontrivial_modes[0]]).reshape(nt, nparticles, 2)
x_from_mode2 = reconstruct_from_mode(q_traj_nt[:, 1], eigenvectors[:, nontrivial_modes[1]]).reshape(nt, nparticles, 2)

# --- Build figure with 2 rows x 3 columns ---
fig = make_subplots(
    rows=2, cols=3,
    shared_yaxes=False,
    row_heights=[0.8, 0.2],
    vertical_spacing=0.05,
    horizontal_spacing=0.08,
    subplot_titles=(
        "Full trajectory",
        "Reconstructed: mode 1",
        "Reconstructed: mode 2",
        "", "", ""
    )
)

# --- Top row: XY particle trajectories ---
datasets = [trajectory, x_from_mode1, x_from_mode2]
for col, data in enumerate(datasets, start=1):
    for i in range(nparticles):
        fig.add_trace(
            go.Scatter(
                x=data[:, i, 0], y=data[:, i, 1],
                mode="lines", line=dict(color=colors[i]),
                name=f"Particle {i}" if col == 1 else None,
                showlegend=(col == 1)
            ),
            row=1, col=col
        )

# --- Bottom row: initial particle positions + bonds ---
for col, data in enumerate(datasets, start=1):
    init_pos = data[0]
    # Particles
    fig.add_trace(
        go.Scatter(
            x=init_pos[:, 0], y=init_pos[:, 1],
            mode="markers",
            marker=dict(size=24, color=colors),
            showlegend=(col == 1)
        ),
        row=2, col=col
    )
    # Bonds
    for (i, j) in bonds:
        fig.add_trace(
            go.Scatter(
                x=[init_pos[i, 0], init_pos[j, 0]],
                y=[init_pos[i, 1], init_pos[j, 1]],
                mode="lines",
                line=dict(color=cb_color if set((i, j)) == {1, 2} else bond_color, width=3),
                showlegend=False
            ),
            row=2, col=col
        )

# --- Build frames for animation ---
frames = []
for step in range(0, nsteps, frame_stride):
    pos = trajectory[step]
    pos_from_mode1 = x_from_mode1[step]
    pos_from_mode2 = x_from_mode2[step]

    frame_data = []

    # Particle trajectories
    for j in range(nparticles):
        frame_data.append(go.Scatter(x=trajectory[:step, j, 0], y=trajectory[:step, j, 1],
                                     mode="lines", line=dict(color=colors[j]), showlegend=False))
    for j in range(nparticles):
        frame_data.append(go.Scatter(x=x_from_mode1[:step, j, 0], y=x_from_mode1[:step, j, 1],
                                     mode="lines", line=dict(color=colors[j]), showlegend=False))
    for j in range(nparticles):
        frame_data.append(go.Scatter(x=x_from_mode2[:step, j, 0], y=x_from_mode2[:step, j, 1],
                                     mode="lines", line=dict(color=colors[j]), showlegend=False))

    # Bottom row: particles + bonds
    for dataset, positions in zip([trajectory, x_from_mode1, x_from_mode2],
                                  [pos, pos_from_mode1, pos_from_mode2]):
        # Particles
        frame_data.append(go.Scatter(x=positions[:, 0], y=positions[:, 1],
                                     mode="markers", marker=dict(size=24, color=colors), showlegend=False))
        # Bonds
        for (i, j) in bonds:
            frame_data.append(go.Scatter(
                x=[positions[i, 0], positions[j, 0]],
                y=[positions[i, 1], positions[j, 1]],
                mode="lines",
                line=dict(color=cb_color if set((i, j)) == {1, 2} else bond_color, width=3),
                showlegend=False
            ))

    frames.append(go.Frame(data=frame_data))

# --- Layout ---
fig.update_layout(
    height=700,
    plot_bgcolor="white",
    xaxis=dict(title="x (full)"), yaxis=dict(title="y (full)"),
    xaxis2=dict(title="x (mode 1)"), yaxis2=dict(title="y (mode 1)"),
    xaxis3=dict(title="x (mode 2)"), yaxis3=dict(title="y (mode 2)"),
    xaxis4=dict(title="x"), yaxis4=dict(title="y"),
    xaxis5=dict(title="x"), yaxis5=dict(title="y"),
    xaxis6=dict(title="x"), yaxis6=dict(title="y"),
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 50, "redraw": True},
                             "fromcurrent": True, "transition": {"duration": 0}}],
             "label": "▶ Play", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": False},
                               "mode": "immediate",
                               "transition": {"duration": 0}}],
             "label": "⏸ Pause", "method": "animate"}
        ],
        "direction": "left", "pad": {"r": 10, "t": 87},
        "showactive": False, "type": "buttons",
        "x": 0.5, "xanchor": "center", "y": 1.4, "yanchor": "top"
    }],
)

fig.frames = frames
fig.show()


**Homework Assignment**

Pick one (or more) of the following projects:
1. Modify the code to handle a 1D systen with N particles. Find a suitable molecule that can be described by your model (e.g. acetylene).
2. Modify the code to handle one of the following 2D systems:
    * Water and ozone (how can you keep the system bent?)
    * Formaldehyde
    * Benzene (only the carbon atoms)  
3. Modify the code to handle a periodic 1D system with 2 particles

For your specific application, please address the following points:
1. Discuss which modes you expect to be able to characterize and if your model is missing any of the modes of the real system.
2. Report the modes, their frequencies and the corresponding atomic displacements. Discuss how these respect/break the symmetry of the system.
3. If you are modeling a real molecule, find in the literature (experimental or computational) some reference values for the different vibrations of the system. Compare the results of your coupled-springs model to the literature, discussing how much of the trends are reproduced and giving a tentative explanation of the agreement/disagreement of your model.  
4. How could you improve your model? Report some possible ideas. We are not asking for very long and detailed explanations, just short sentences and keywords.

NOTE: It is not necessary that the modified code produces an animation, but you should be able to visualize the results of the simulation in some way.